# Orchid Classification using CNN (MobileNetV2) + K-Fold + Confusion Matrix

---
# 1. Install TensorFlow 2.2
---

In [ ]:
!pip install tensorflow==2.2

In [ ]:
import tensorflow as tf
from tensorflow import keras

print("TensorFlow Version :", tf.__version__)
print("Keras Version      :", keras.__version__)

In [ ]:
!pip list --version

---
# 2. Data Preprocessing
---

## 2-1. Load Image Data as Array

In [ ]:
import os
import cv2
import random
import numpy as np

img_size_224p = 128 

path_train  = '../input/orchid-genus/orchid-genus/train'
path_test   = '../input/orchid-genus/orchid-genus/test'
categories  = ['cattleya', 'dendrobium', 'oncidium', 'phalaenopsis', 'vanda']

def create_data_img(folder_path):
    imageData = []
    for category in categories:
        path = os.path.join(folder_path, category)
        class_num = categories.index(category) # Take the Label as the Index
        for img in os.listdir(path):
            img_array   = cv2.imread(os.path.join(path, img)) 
            img_convert = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
            img_resize  = cv2.resize(img_convert, (img_size_224p, img_size_224p))
            imageData.append([img_resize, class_num])
    
    return imageData

dataTrain   = create_data_img(path_train)
dataTest    = create_data_img(path_test)

# Shuffle the Train Data (if don't shuffle, the Train Data will be sorted by Labels)
random.seed(10) # 10 as the Shuffle Index, so that when re-running the program, the results of the shuffle are the same
random.shuffle(dataTrain)

## 2-2. Get Features (X) & Labels (y)

In [ ]:
# X for Features & y for Labels
X_train, y_train, X_test, y_test = [], [], [], []

for features, label in dataTrain:
    X_train.append(features)
    y_train.append(label)

for features, label in dataTest:
    X_test.append(features)
    y_test.append(label)

# -1 in reshape, means to let Numpy define the appropriate data dimensions
X_train = np.array(X_train).reshape(-1, img_size_224p, img_size_224p, 3)
y_train = np.asarray(y_train)
X_test  = np.array(X_test).reshape(-1, img_size_224p, img_size_224p, 3)
y_test  = np.asarray(y_test)

print("X_train :", X_train.shape)
print("y_train :", y_train.shape)
print("X_test  :", X_test.shape)
print("y_test  :", y_test.shape)

## 2-3. Features (X) : Feature Scaling

In [ ]:
print("Array of X_train :\n\n", X_train[0]) # Take the first data for example
print("\nArray of X_test  :\n\n", X_test[0])

def prep_pixels(train, test):
    # Convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # Normalize (feature scaling) to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # Return normalized images
    return train_norm, test_norm

X_train_norm, X_test_norm = prep_pixels(X_train, X_test)

print("\nArray of X_train_norm :\n\n", X_train_norm[0])
print("\nArray of X_test_norm  :\n\n", X_test_norm[0])

## 2-4. Labels (y) : One Hot Encoding

In [ ]:
from keras.utils import to_categorical

print("Array of y_train :", y_train)
print("Array of y_test  :", y_test)

# One Hot Encode target values
y_train_encode = to_categorical(y_train)
y_test_encode  = to_categorical(y_test)

print("\nArray of y_train_encode :\n\n", y_train_encode)
print("\nArray of y_test_encode :\n\n", y_test_encode)

## 2-5. Plot the Dataset

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

nrows = 5 
ncols = 5  
hspace = 0
wspace = 0
fig, ax = plt.subplots(nrows, ncols, figsize=(10, 10))    
fig.subplots_adjust(hspace, wspace)

for i in range(nrows):
    for j in range(ncols):
        temp = i*ncols+j                # Index looping
        ax[i,j].imshow(X_train[temp])   # Show Features/images
        if y_train[temp] == 0:
            judul = "cattleya"
        elif y_train[temp] == 1:
            judul = "dendrobium"
        elif y_train[temp] == 2:
            judul = "oncidium"
        elif y_train[temp] == 3:
            judul = "phalaenopsis"
        elif y_train[temp] == 4:
            judul = "vanda"
        ax[i,j].set_title(judul)        # Show Labels
        ax[i,j].axis('off')             # Hide axis
plt.show()

## 2-6. Clean up Useless Data (RAM Cleaner)

In [ ]:
import gc     # Gabage Collector for cleaning deleted data from memory

del dataTrain
del dataTest
del X_train
del X_test
#del y_train  # Used later for Confusion Matrix
#del y_test   # Used later for Confusion Matrix

gc.collect()

## 2-7. The Final Data to be used on CNN

In [ ]:
print("X_train_norm     :", X_train_norm.shape)
print("y_train_encode   :", y_train_encode.shape)
print("X_test_norm      :", X_test_norm.shape)
print("y_test_encode    :", y_test_encode.shape)

---
# 3. Build CNN Architecture: MobileNetV2
---

## 3-1. Load a Pretrained MobileNetV2 Model

In [ ]:
from keras.applications import MobileNetV2
from keras.utils import plot_model

'''
Important Notes:

weights='imagenet'            The initial weights are filled directly with the "optimal" weight of the imagenet (pre-trained).
weights=None                  The initial weight are filled with a random value (in case: training from scratch).
include_top=False             Cut the head (top) of mobilenetv2 architecture, so that it can be modified according to the label used (in case: orchid).
conv_base.trainable=False     Can only be used if weights="imagenet", this means the weight in the feature extractor will be frozen,
                              it will not be updated during training, in other words, the extractor feature is only used.
'''

conv_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size_224p, img_size_224p, 3))
conv_base.trainable = False
conv_base.summary()
plot_model(conv_base, to_file='model.png', show_shapes=True, show_layer_names=False, rankdir='TB', expand_nested=False, dpi=80)

## 3-2. Modified a MobileNetV2 Architecture

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from keras.optimizers import Adam

def define_model_mobilenetv2():
    model = Sequential()
    model.add(conv_base)                        # The Feature Extractor uses a Pre-trained Model
    model.add(GlobalAveragePooling2D())
    model.add(Dense(5, activation='softmax'))   # This means that in the Hidden Layer there are 5 Neurons (5 Orchid Labels)
                                                
    
    # Compile Model
    opt = Adam(lr=0.0001)                      
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy is used because
    return model                                                                        # of the Multi-Class Classification problem

# Clean the Previous Model (retraining needs)
if "model" in globals(): # Check if the Model Variables exist
  del model
  gc.collect()

model = define_model_mobilenetv2()
model.summary()
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=False, rankdir='TB', expand_nested=False, dpi=80)

---
# 4. OPTION 1: Training with K-Fold Cross Validation
---

## 4-1. Training with 5-Fold

In [ ]:
%%time

import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit

def evaluate_model(dataX, dataY, n_folds=5):  
    epochs = 10                               
    batch_size = 64                           

    scores, histories = list(), list()
    kfold = KFold(n_folds, shuffle=True, random_state=1) 

    i = 0
    # Enumerate splits
    for train_ix, val_ix in kfold.split(dataX):
        i = i+1
        model = define_model_mobilenetv2() 
        trainX, trainY, valX, valY = dataX[train_ix], dataY[train_ix], dataX[val_ix], dataY[val_ix]  
        history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_data=(valX, valY), verbose=1) 
        loss, acc = model.evaluate(valX, valY, verbose=0) # Evaluate Model
        print('\nFold ' + str(i) + ' Accuracy = %.3f' % (acc * 100.0))
        print('Fold ' + str(i) + ' Loss = %.3f' % (loss) + '\n')
        scores.append(acc) # Append Scores
        histories.append(history) # Append Histories

        #----------------------------- Additional -----------------------------#

        model.save("model_fold_" + str(i) + ".h5")  # Save Model as h5
        model_csv = pd.DataFrame(history.history)   # Save Model Report to csv
        csv_file = "model_fold_" + str(i) + ".csv"
        with open(csv_file, mode="w") as f:
          model_csv.to_csv(f)
        
        # Clean the RAM for every Fold
        del trainX
        del trainY
        del valX
        del valY
        del model
        gc.collect()

    return scores, histories
    
scores, histories = evaluate_model(X_train_norm, y_train_encode)

## 4-2. Plot the Graphs of Training & Validation Results (Combine)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def summarize_diagnostics_combine(histories):
    plt.figure(figsize=(10,10))
    
    for i in range(len(histories)):
        # Loss Plot
        plt.subplot(211) # 2 rows, 1 column, 1st index
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.plot(histories[i].history['loss'], color='blue', marker='.', label='train')
        plt.plot(histories[i].history['val_loss'], color='orange', marker='.', label='test')
        plt.legend(['train', 'validation'], loc='upper right')
        
        # Accuracy Plot
        plt.subplot(212) # 2 rows, 1 column, 2nd index
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.plot(histories[i].history['accuracy'], color='blue', marker='.', label='train')
        plt.plot(histories[i].history['val_accuracy'], color='orange', marker='.', label='test')
        plt.legend(['train', 'validation'], loc='lower right')
    plt.show()

summarize_diagnostics_combine(histories)

## 4-2. Plot the Graphs of Training & Validation Results (Single)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def summarize_diagnostics_single(histories):
    for i in range(len(histories)):
        plt.figure(figsize=(16,6))

        # Loss Plot
        plt.subplot(221) # 2 rows, 2 column, 1st index
        plt.title('Fold ' + str(i+1))
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.plot(histories[i].history['loss'], color='blue', marker='.', label='train')
        plt.plot(histories[i].history['val_loss'], color='orange', marker='.', label='test')
        plt.legend(['train', 'validation'], loc='upper right')

        # Accuracy Plot
        plt.subplot(222) # 2 rows, 2 column, 2nd index
        plt.title('Fold ' + str(i+1))
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.plot(histories[i].history['accuracy'], color='blue', marker='.', label='train')
        plt.plot(histories[i].history['val_accuracy'], color='orange', marker='.', label='test')
        plt.legend(['train', 'validation'], loc='lower right')
        plt.show()

summarize_diagnostics_single(histories)

## 4-3. Mean & Standard Deviation Scores

In [ ]:
from numpy import mean
from numpy import std

def summarize_performance(scores):
    print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
    plt.boxplot(scores)
    plt.show()

# Summarize estimated performance
summarize_performance(scores)

---
# 4. OPTION 2: Training without K-Fold Cross Validation
---

## 4-1. Training without K-Fold

In [ ]:
%%time

import pandas as pd

epochs = 10       
batch_size = 64   

model = define_model_mobilenetv2() # Define Model: Using MobileNetV2 which has been modified before
history = model.fit(X_train_norm, y_train_encode, epochs=epochs, batch_size=batch_size, verbose=1) # Fit model

## 4-2. Plot the Graphs of Training Results

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 3))
ax.plot(history.history['accuracy'], 'og', linestyle='dashed')
#ax.plot(history.history['val_accuracy'])
ax.set_ylabel('Accuracy')
ax.set_xlabel('Epoch')
#ax.legend(['train', 'val'], loc='lower right')
plt.show()

fig, ax = plt.subplots(1, 1, figsize=(8, 3))
ax.plot(history.history['loss'], 'ob', linestyle='dashed')
#ax.plot(history.history['val_loss'])
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
#ax.legend(['train', 'val'], loc='upper right')
plt.show()

## 4-3. Save Model as H5 & CSV

In [ ]:
model.save("model_without_kfold.h5")       # Save Model as h5
model_csv = pd.DataFrame(history.history)  # Save Model Report to csv
csv_file = "model_without_kfold.csv"
with open(csv_file, mode="w") as f:
  model_csv.to_csv(f)

---
# 5. Testing Our CNN Model
---

## 5-1. Load Selected CNN Model

In [ ]:
# Clean the Previous Model (RAM Cleaner)
if "model" in globals():
  del model
  gc.collect()

# Load Model (Enter Path of Selected Model)
from keras.models import load_model
model = load_model('./model_fold_5.h5') 
#model.summary()

## 5-2. Testing Model with Test Data

In [ ]:
from sklearn.preprocessing import LabelBinarizer

if "encoder" in globals(): # RAM Cleaner
  del encoder
  del y_train_encode_new
  del y_test_encode_new
  del pred
  del prediksi
  del pred_label
  del true_label
  gc.collect()

encoder             = LabelBinarizer() # Encoding Labels (y) in different ways, for Confusion Matrix purposes
y_train_encode_new  = encoder.fit_transform(y_train)
y_test_encode_new   = encoder.fit_transform(y_test)

pred        = model.predict(X_test_norm.astype('float32'), verbose=0)
prediksi    = np.argmax(pred, axis=-1) # Try -> predict.shape -> (800, 5) -> axis = -1 it will get that value 5 (number of Orchid Labels)

pred_label  = model.predict_classes(X_test_norm, batch_size=64, verbose=0)  # Prediction Result Label
true_label  = np.argmax(y_test_encode_new, axis=-1)                         # Actual Label (in Dataset)

print("Predict Label :", pred_label)
print("Actual Label  :", true_label, "\n")

loss, acc = model.evaluate(X_test_norm, y_test_encode_new, verbose=1)

## 5-3. Evaluate Model with Confusion Matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

target_names = ['cattleya', 'dendrobium', 'oncidium', 'phalaenopsis', 'vanda']
cmatrix = confusion_matrix(true_label, pred_label)
creport = classification_report(true_label, prediksi, target_names=target_names)

print("Accuracy : {:.3f}%".format(acc*100))
print("Loss     : {:.3f}".format(loss))

print("\nClassification Report :\n")
print(creport)

fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(cmatrix, cmap="crest_r", annot=True, fmt='.4g', linewidths=2, linecolor='white', cbar=False, ax=ax)
# cmap options: rocket, mako, flare, crest, magma, viridis, rocket_r, cubehelix, seagreen, Blues, ...

ax.set_title('Confusion Matrix', fontsize=18, pad=24)
ax.set_xticklabels(labels=target_names, fontsize=12)
ax.set_yticklabels(labels=target_names, fontsize=12)

plt.xlabel("(y) Predict Label", fontsize=16, color="darkgreen", labelpad=24)
plt.ylabel("(y) Actual Label", fontsize=16, color="darkgreen", labelpad=24)
plt.show()

## 5-4. Testing Model with Test Data from Internet

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# Load and Prepare the Image
def load_image(filename):
    img = load_img(filename, target_size=(img_size_224p, img_size_224p))
    plt.imshow(img)
    plt.axis("off");
    img = img_to_array(img)
    img = img.reshape(-1, img_size_224p, img_size_224p, 3)
    img = img.astype('float32')
    img = img / 255.0
    return img

# Load an Image and Predict the Class/Label
def run_example(new_data_path):
    # Load the Image
    img = load_image(new_data_path)
    # Load Model
    model = load_model('./model_fold_5.h5') 
    # Predict the Class/Label
    result = model.predict_classes(img) # OPTION 1
    #result = model.predict(img)        # OPTION 2
    if result[0] == 0:
        print("\nPredict Label: Cattleya")
    elif result[0] == 1:
        print("\nPredict Label: Dendrobium")
    elif result[0] == 2:
        print("\nPredict Label: Oncidium")
    elif result[0] == 3:
        print("\nPredict Label: Phalaenopsis")
    elif result[0] == 4:
        print("\nPredict Label: Vanda")

In [ ]:
# Get image data directly from the internet
#!wget -O 'new_test_data.jpg' 'https://lumencms.blob.core.windows.net/media-generated/538/_L2A0849-ANSU-VANDA-Vanda-Terra-2-600-600.jpg'

# Get image data from the dataset
new_data_path = '../input/orchid-genus/orchid-genus/inet/dendrobium/D1.jpg' 
run_example(new_data_path)